In [ ]:
!pip install --upgrade llama-index
!pip install --upgrade llama-index-vector-stores-weaviate
%pip install llama-index-embeddings-voyageai
%pip install llama-index llama-index-llms-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.8/195.8 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.2/390.2 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.4/325.4 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.

In [ ]:
import pandas as pd
import os
from llama_index.embeddings.voyageai import VoyageEmbedding
from llama_index.llms.openai import OpenAI
from google.colab import userdata
from llama_index.core import Settings
import weaviate
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from llama_index.core.response.notebook_utils import display_response
from llama_index.core import StorageContext
from llama_index.core.retrievers import VectorIndexAutoRetriever
from llama_index.core.vector_stores.types import MetadataInfo, VectorStoreInfo
from llama_index.llms.openai import OpenAI
from llama_index.core.prompts import PromptTemplate
from llama_index.core.postprocessor import SentenceTransformerRerank
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.response_synthesizers import get_response_synthesizer
import pandas as pd
from llama_index.core.vector_stores.types import MetadataFilters, MetadataFilter
import pandas as pd
from tqdm import tqdm
import pickle
import json
from pathlib import Path
from google.colab import drive
import re

In [ ]:
api_key = userdata.get('OPENAI_API_KEY')
voyage_api_key = userdata.get('VOYAGE_API_KEY')

In [ ]:
model_name = "voyage-large-2-instruct"
embed_model = VoyageEmbedding(
    model_name=model_name, voyage_api_key=voyage_api_key
)
embeddings = embed_model.get_query_embedding("What is llamaindex?")

In [ ]:
llm = OpenAI(model="gpt-4o-mini", temperature=0.1, logprobs=None, default_headers={})

In [ ]:
resp = llm.complete("Paul Graham is ")
resp

CompletionResponse(text='Paul Graham is an influential computer scientist, entrepreneur, and venture capitalist, best known for his work in the field of programming languages and for co-founding the startup accelerator Y Combinator. He is also a prominent essayist, having written extensively on topics related to startups, technology, and philosophy. Graham is known for his advocacy of Lisp programming language and for his essays that provide insights into entrepreneurship, innovation, and the tech industry. His work has had a significant impact on the startup ecosystem, particularly in Silicon Valley.', additional_kwargs={}, raw=ChatCompletion(id='chatcmpl-AduAs6I2T2yZ8TRUc59y9KyqCyzaw', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Paul Graham is an influential computer scientist, entrepreneur, and venture capitalist, best known for his work in the field of programming languages and for co-founding the startup accelerator Y Combin

In [ ]:
Settings.embed_model = embed_model
Settings.llm = OpenAI(model="gpt-4o-mini", temperature=0.1)

In [ ]:
cluster_url = "https://ne9xrbsctchldvd4jhhfq.c0.us-east1.gcp.weaviate.cloud"
api_key = userdata.get('WEAVITE_API_KEY')

In [ ]:
client = weaviate.connect_to_wcs(
    cluster_url=cluster_url,
    auth_credentials=weaviate.auth.AuthApiKey(api_key),
)

In [ ]:
vector_store = WeaviateVectorStore(
    weaviate_client=client,
    index_name="RestaurantDataRag"
)

In [ ]:
loaded_index = VectorStoreIndex.from_vector_store(vector_store)

In [ ]:
query_engine = loaded_index.as_query_engine(similarity_top_k=2)

In [ ]:
response = query_engine.query("How is the food at Iron Hill?")
display_response(response)

**`Final Response:`** The food at Iron Hill Brewery & Restaurant is described as just OK, with a massive menu offering a variety of options. However, specific dishes like the burger and seared tuna salad received mixed reviews, with comments on the burger being average and the tuna lacking flavor.

In [ ]:
vector_store_info = VectorStoreInfo(
    content_info="Restaurant reviews and ratings",
    metadata_info=[
        MetadataInfo(
            name="restaurant_name",
            type="str",
            description="Name of the restaurant"
        ),
        MetadataInfo(
            name="business_id",
            type="str",
            description="Unique identifier for the restaurant"
        ),
        MetadataInfo(
            name="review_date",
            type="str",
            description="Date when the review was written"
        )
    ],
)

In [ ]:
retriever = VectorIndexAutoRetriever(
    loaded_index,
    llm=OpenAI(model="gpt-4o-mini"),
    vector_store_info=vector_store_info,
    similarity_top_k=4,
    vector_store_query_mode="hybrid",
    alpha=0.5,
    verbose=True
)

In [ ]:
rerank = SentenceTransformerRerank(
    top_n=2,
    model="BAAI/bge-reranker-base"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [ ]:
qa_prompt_tmpl_str = """\
Context information is below.
---------------------
{context_str}
---------------------

Given the restaurant reviews provided in the context, answer the user query.
Focus on specific details from the reviews and provide examples when relevant.

Some examples are given below.

Query: What do people say about the service at Restaurant X?
Answer: Based on the reviews, Restaurant X's service is highly rated. Multiple reviewers mentioned their server "John" who was attentive and professional.

Query: What are the most recommended dishes?
Answer: According to the reviews, the most recommended dishes are the seafood pasta and the chocolate dessert. Three different reviewers specifically praised these items.

Query: {query_str}
Answer: """

In [ ]:
qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)
response_synthesizer = get_response_synthesizer(
    text_qa_template=qa_prompt_tmpl
)

In [ ]:
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[rerank],
)

In [ ]:
sample_query = "Does Il Fiore Restaurant serve Caribbean food?"
response = query_engine.query(sample_query)
print(response)

Using query str: restaurant reviews for Il Fiore Restaurant
Using filters: [('restaurant_name', '==', 'Il Fiore Restaurant')]
No, Il Fiore Restaurant does not serve Caribbean food. It specializes in Italian cuisine, particularly authentic dishes that reflect Sicilian traditions. One reviewer mentioned that the food is prepared differently from more Americanized Italian restaurants, indicating a focus on traditional Italian flavors and techniques.


In [ ]:
df_qs = pd.read_csv('/content/test_final.csv')

In [ ]:
df_qs = df_qs[['business_id', 'name', 'question']]

In [ ]:
drive.mount('/content/drive')
save_dir = '/content/drive/My Drive/restaurant_qa_results'
os.makedirs(save_dir, exist_ok=True)

Mounted at /content/drive


In [ ]:
for business_id, group in df_qs.groupby('business_id'):

    name = group['name'].iloc[0]
    restaurant_results = []

    filters = MetadataFilters(
        filters=[
            MetadataFilter(
                key="restaurant_name",
                value=name
            )
        ]
    )

    retriever = VectorIndexAutoRetriever(
        loaded_index,
        llm=OpenAI(model="gpt-4o-mini"),
        vector_store_info=vector_store_info,
        similarity_top_k=4,
        vector_store_query_mode="hybrid",
        alpha=0.5,
        verbose=True,
        filter_dict={"restaurant_name": name}
    )

    restaurant_query_engine = RetrieverQueryEngine(
        retriever=retriever,
        response_synthesizer=response_synthesizer,
        node_postprocessors=[rerank],
    )

    filename = os.path.join(save_dir, f'{business_id}_qa.json')
    restaurant_results = []

    print(f"\nProcessing {name}")
    progress_bar = tqdm(total=len(group), desc="Processing questions")

    for _, row in group.iterrows():
        try:
            response = restaurant_query_engine.query(row['question'])
            retrieved_context = []
            for node in response.source_nodes:
              retrieved_context.append(node.node.text)
            result = {
                'business_id': business_id,
                'restaurant_name': name,
                'question': row['question'],
                'response': str(response),
                'status': 'success'
            }
        except Exception as e:
            result = {
                'business_id': business_id,
                'restaurant_name': name,
                'question': row['question'],
                'response': f"Error: {str(e)}",
                'context': ' | '.join(retrieved_context)
            }

        restaurant_results.append(result)
        progress_bar.update(1)

    # Save results for this restaurant
    with open(filename, 'w') as f:
        json.dump(restaurant_results, f, indent=4)

    progress_bar.close()
    print(f"Saved results for {name}")


Processing Keg & Kitchen


Processing questions:   0%|          | 0/70 [00:00<?, ?it/s]

Using query str: salads
Using filters: [('restaurant_name', '==', 'Keg & Kitchen')]


Processing questions:   1%|▏         | 1/70 [00:04<04:56,  4.30s/it]

Using query str: food service at Keg & Kitchen
Using filters: []


...

Using query str: service consistency
Using filters: [('restaurant_name', '==', 'Tortilla Press')]


Processing questions:  99%|█████████▊| 67/68 [03:49<00:03,  3.05s/it]

Using query str: Does Tortilla Press accept credit cards?
Using filters: []


Processing questions: 100%|██████████| 68/68 [03:52<00:00,  3.42s/it]

Saved results for Tortilla Press


In [ ]:
def read_all_json_files(directory):
    """
    Read all JSON files from a directory and combine them into a single DataFrame.

    Args:
        directory (str): Path to the directory containing JSON files

    Returns:
        pd.DataFrame: Combined DataFrame from all JSON files
    """
    dir_path = Path(directory)

    dfs = []

    for file_path in dir_path.glob('*.json'):
        try:
            df = pd.read_json(file_path)
            df['source_file'] = file_path.name
            dfs.append(df)
        except Exception as e:
            print(f"Error reading {file_path}: {str(e)}")

    if not dfs:
        raise ValueError(f"No JSON files found in {directory}")

    combined_df = pd.concat(dfs, ignore_index=True)
    return combined_df

In [ ]:
try:
    df = read_all_json_files(save_dir)
    print(f"Successfully loaded {len(df)} rows from JSON files")
except Exception as e:
    print(f"Error: {str(e)}")

Successfully loaded 3095 rows from JSON files


In [ ]:
df = df[['business_id', 'question', 'response']]

In [ ]:
df_test = pd.read_csv('/content/test_final.csv')
final_df = pd.merge(df, df_test, on=['question'], how='left')

In [ ]:
def normalize_text(text):
    """
    Normalize text by:
    1. Converting to lowercase
    2. Removing extra whitespace
    3. Removing punctuation
    4. Standardizing special characters
    """
    # Convert to lowercase
    text = text.lower()

    # Remove punctuation and special characters, keeping only alphanumeric and spaces
    text = re.sub(r'[^a-z0-9\s]', '', text)

    # Replace multiple spaces with single space and trim
    text = ' '.join(text.split())

    return text

def calculate_exact_match(row):
    """
    Calculate Exact Match score (0 or 1) for a single pair of answers
    """
    actual = normalize_text(row['answer'])
    predicted = normalize_text(row['response'])
    return 1 if actual == predicted else 0

def evaluate_exact_match(df):
    """
    Calculate Exact Match scores for the entire dataset
    Returns:
    - DataFrame with EM scores
    - Overall EM accuracy
    """
    # Create copy of dataframe to avoid modifying original
    df_eval = df.copy()

    # Calculate EM score for each row
    df_eval['exact_match_score'] = df_eval.apply(calculate_exact_match, axis=1)

    # Calculate overall accuracy
    overall_em = df_eval['exact_match_score'].mean()

    return df_eval, overall_em

In [ ]:
df_with_scores, overall_accuracy = evaluate_exact_match(final_df)

In [ ]:
overall_accuracy

0.0

In [ ]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
import string
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from collections import Counter

class F1Evaluator:
    def __init__(self):
        # Download required NLTK data
        try:
            nltk.download('punkt', quiet=True)
        except:
            pass

    def preprocess_text(self, text):
        """
        Preprocess text by:
        1. Converting to lowercase
        2. Removing punctuation
        3. Tokenizing into words
        """
        # Convert to lowercase
        text = text.lower()

        # Remove punctuation
        text = text.translate(str.maketrans("", "", string.punctuation))

        # Tokenize
        tokens = word_tokenize(text)

        # Remove empty tokens
        tokens = [token for token in tokens if token.strip()]

        return tokens

    def calculate_f1_score(self, actual, predicted):
        """
        Calculate F1 score between actual and predicted answers
        """
        # Tokenize both answers
        actual_tokens = self.preprocess_text(actual)
        predicted_tokens = self.preprocess_text(predicted)

        # Convert to Counter objects for easy intersection
        actual_counter = Counter(actual_tokens)
        predicted_counter = Counter(predicted_tokens)

        # Calculate intersection
        intersection = sum((actual_counter & predicted_counter).values())

        # Calculate precision and recall
        precision = intersection / len(predicted_tokens) if predicted_tokens else 0
        recall = intersection / len(actual_tokens) if actual_tokens else 0

        # Calculate F1 score
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        return {
            'f1_score': f1,
            'precision': precision,
            'recall': recall,
            'actual_tokens': len(actual_tokens),
            'predicted_tokens': len(predicted_tokens),
            'matching_tokens': intersection
        }

def evaluate_qa_f1(df):
    """
    Evaluate QA pairs using F1 score
    """
    evaluator = F1Evaluator()

    # Calculate scores for each row
    results = []
    for _, row in df.iterrows():
        score_dict = evaluator.calculate_f1_score(row['answer'], row['response'])
        results.append(score_dict)

    # Add scores to DataFrame
    df_scored = df.copy()
    for metric in ['f1_score', 'precision', 'recall', 'matching_tokens']:
        df_scored[metric] = [r[metric] for r in results]

    # Calculate overall scores
    overall_scores = {
        'f1_score': np.mean([r['f1_score'] for r in results]),
        'precision': np.mean([r['precision'] for r in results]),
        'recall': np.mean([r['recall'] for r in results])
    }

    return df_scored, overall_scores

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
df_with_scores, overall_scores = evaluate_qa_f1(final_df)

In [ ]:
overall_scores

{'f1_score': 0.273654168980317,
 'precision': 0.21683314288996897,
 'recall': 0.5645315440480198}

In [ ]:
import torch
from transformers import RobertaTokenizer, RobertaModel
import numpy as np
from tqdm import tqdm
import pandas as pd

class BERTScoreEvaluator:
    def __init__(self, model_name='roberta-large', layer=17, batch_size=32):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.tokenizer = RobertaTokenizer.from_pretrained(model_name)
        self.model = RobertaModel.from_pretrained(model_name).to(self.device)
        self.layer = layer
        self.batch_size = batch_size

    def get_embeddings(self, text_batch):
        """Extract embeddings from specified layer for a batch of text"""
        # Tokenize and move to device
        inputs = self.tokenizer(text_batch,
                              padding=True,
                              truncation=True,
                              return_tensors="pt").to(self.device)

        # Get model outputs
        with torch.no_grad():
            outputs = self.model(**inputs, output_hidden_states=True)

        # Get embeddings from specified layer
        layer_embeddings = outputs.hidden_states[self.layer]

        # Create attention mask for proper averaging
        attention_mask = inputs['attention_mask'].unsqueeze(-1)

        # Apply mask and compute mean
        masked_embeddings = layer_embeddings * attention_mask
        sum_embeddings = torch.sum(masked_embeddings, dim=1)
        sum_mask = torch.sum(attention_mask, dim=1)
        embeddings = sum_embeddings / sum_mask

        return embeddings.cpu().numpy()

    def calculate_bertscore(self, references, candidates):
        """Calculate BERTScore for batches of text"""
        scores = []

        # Process in batches
        for i in range(0, len(references), self.batch_size):
            batch_refs = references[i:i + self.batch_size]
            batch_cands = candidates[i:i + self.batch_size]

            # Get embeddings
            ref_embeddings = self.get_embeddings(batch_refs)
            cand_embeddings = self.get_embeddings(batch_cands)

            # Normalize embeddings
            ref_embeddings = ref_embeddings / np.linalg.norm(ref_embeddings, axis=1)[:, None]
            cand_embeddings = cand_embeddings / np.linalg.norm(cand_embeddings, axis=1)[:, None]

            # Calculate cosine similarity
            batch_scores = np.sum(ref_embeddings * cand_embeddings, axis=1)
            scores.extend(batch_scores)

        return np.array(scores)

def evaluate_qa_bertscore(df):
    """
    Evaluate QA pairs using BERTScore
    """
    # Initialize evaluator
    evaluator = BERTScoreEvaluator()

    # Get lists of answers
    actual_answers = df['answer'].tolist()
    predicted_answers = df['response'].tolist()

    # Calculate scores
    print("Calculating BERTScores...")
    bertscores = evaluator.calculate_bertscore(actual_answers, predicted_answers)

    # Add scores to DataFrame
    df_scored = df.copy()
    df_scored['bertscore'] = bertscores

    # Calculate overall score
    overall_score = np.mean(bertscores)

    return df_scored, overall_score

In [ ]:
df_with_scores, overall_bertscore = evaluate_qa_bertscore(final_df)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Calculating BERTScores...


In [ ]:
overall_bertscore

0.9792175

In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

class BiEncoderEvaluator:
    def __init__(self, model_name='all-MiniLM-L6-v2'):
        """
        Initialize the Bi-Encoder evaluator with specified model
        """
        self.model = SentenceTransformer(model_name)

    def calculate_similarity(self, actual_answers, predicted_answers):
        """
        Calculate cosine similarity between actual and predicted answer embeddings
        """
        # Generate embeddings for both sets of answers
        actual_embeddings = self.model.encode(actual_answers)
        predicted_embeddings = self.model.encode(predicted_answers)

        # Calculate cosine similarity for each pair
        similarities = [cosine_similarity(
            actual_emb.reshape(1, -1),
            pred_emb.reshape(1, -1)
        )[0][0] for actual_emb, pred_emb in zip(actual_embeddings, predicted_embeddings)]

        return similarities

def evaluate_qa_biencoder(df):
    """
    Evaluate QA pairs using Bi-Encoder score
    """
    # Initialize evaluator
    evaluator = BiEncoderEvaluator()

    # Get lists of answers
    actual_answers = df['answer'].tolist()
    predicted_answers = df['response'].tolist()

    # Calculate similarities
    scores = evaluator.calculate_similarity(actual_answers, predicted_answers)

    # Add scores to DataFrame
    df_scored = df.copy()
    df_scored['biencoder_score'] = scores

    # Calculate overall score
    overall_score = np.mean(scores)

    return df_scored, overall_score


In [ ]:
df_with_scores, overall_score = evaluate_qa_biencoder(final_df)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
overall_score

0.6855832

In [ ]:
!pip install sentence-transformers

In [ ]:
import pandas as pd
import numpy as np
from typing import List, Dict, Union

def evaluate_semantic_relevance(df: pd.DataFrame,
                              question_col: str = 'question',
                              answer_col: str = 'answer') -> pd.DataFrame:
    """
    Evaluate semantic relevance between questions and answers using embeddings.

    Args:
        df: DataFrame containing questions and answers
        question_col: Name of question column
        answer_col: Name of answer column

    Returns:
        DataFrame with added semantic similarity scores
    """
    try:
        from sentence_transformers import SentenceTransformer
    except ImportError:
        raise ImportError("Please install sentence-transformers: pip install sentence-transformers")

    try:
        model = SentenceTransformer('all-MiniLM-L6-v2')
        result_df = df.copy()
        if question_col not in df.columns or answer_col not in df.columns:
            raise KeyError(f"Missing required columns. Needed: {question_col}, {answer_col}")
        print("Generating question embeddings...")
        question_embeddings = model.encode(df[question_col].fillna('').tolist(),
                                        show_progress_bar=True)

        print("Generating answer embeddings...")
        answer_embeddings = model.encode(df[answer_col].fillna('').tolist(),
                                      show_progress_bar=True)

        print("Calculating similarities...")
        similarities = [
            float(np.dot(q_emb, a_emb) / (np.linalg.norm(q_emb) * np.linalg.norm(a_emb)))
            for q_emb, a_emb in zip(question_embeddings, answer_embeddings)
        ]

        result_df['semantic_similarity'] = similarities
        result_df['relevance_rating'] = pd.cut(
            result_df['semantic_similarity'],
            bins=[-1, 0.3, 0.5, 0.7, 1],
            labels=['Low', 'Moderate', 'Good', 'Excellent']
        )

        print("Evaluation complete!")
        return result_df

    except Exception as e:
        print(f"Error during semantic evaluation: {str(e)}")
        raise

def print_evaluation_summary(df: pd.DataFrame) -> None:
    """
    Print a summary of the semantic evaluation results.

    Args:
        df: DataFrame with semantic_similarity scores
    """
    try:
        if 'semantic_similarity' not in df.columns:
            raise KeyError("DataFrame does not contain semantic_similarity scores. Run evaluate_semantic_relevance first.")

        print("\nEvaluation Summary:")
        print("-" * 50)
        print(f"Total examples evaluated: {len(df)}")
        print(f"Average similarity score: {df['semantic_similarity'].mean():.3f}")
        print(f"Median similarity score: {df['semantic_similarity'].median():.3f}")
        print("\nRelevance Distribution:")
        print(df['relevance_rating'].value_counts().sort_index())

        # Print example from each category
        print("\nExample from each category:")
        for category in ['Low', 'Moderate', 'Good', 'Excellent']:
            category_df = df[df['relevance_rating'] == category]
            if not category_df.empty:
                example = category_df.iloc[0]
                print(f"\n{category} relevance example (score: {example['semantic_similarity']:.3f}):")
                print(f"Question: {example['question']}")
                print(f"Answer: {example['response']}")

    except Exception as e:
        print(f"Error during summary printing: {str(e)}")
        raise

if __name__ == "__main__":

    results_df = evaluate_semantic_relevance(final_df)
    print_evaluation_summary(results_df)

Generating question embeddings...


Batches:   0%|          | 0/97 [00:00<?, ?it/s]

Generating answer embeddings...


Batches:   0%|          | 0/97 [00:00<?, ?it/s]

Calculating similarities...
Evaluation complete!

Evaluation Summary:
--------------------------------------------------
Total examples evaluated: 3079
Average similarity score: 0.685
Median similarity score: 0.718

Relevance Distribution:
relevance_rating
Low           133
Moderate      279
Good          981
Excellent    1686
Name: count, dtype: int64

Example from each category:

Low relevance example (score: 0.233):
Question: How does Dolsot House handle first-time Korean cuisine customers?
Answer: Based on the context provided, there are no specific reviews detailing how Dolsot House handles first-time Korean cuisine customers. However, the restaurant is noted for offering "excellent options for Korean" and "authentic Korean foods," which suggests that they likely provide a welcoming atmosphere for newcomers to Korean cuisine. It would be reasonable to assume that the staff may be knowledgeable and willing to guide first-time diners through the menu, helping them choose dishes that